# A Guide to TF Layers:Building a Convolutional Neural Network

参考链接：[A Guide to TF Layers:Building a Convolutional Neural Network](https://www.tensorflow.org/tutorials/layers)

Tensorflow的layer模块提供高级API使得构建神经网络变得简单。layer提供方法method简化全连接层和卷积层的创建，增加激活函数activation functions，应用弃权规则化dropout regularization.本教程中，你将学习如何使用layers模块建立卷积神经网络模型来识别MNIST data set中的手写数字。

MNIST dataset包含0-9手写数字对应的60,000个训练数据（图片）和10,000个测试数据，图片是28*28像素单色图片。

## Getting Started

首先构建Tensorflow程序的基本结构，创建名为`cnn_mnist.py`的文件，并在文件中添加如下代码：
```
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

if __name__ == "__main__":
  tf.app.run()
  ```
  
如果你学完了本教程，你将添加代码以构建、训练和评估卷积神经网络。完整最终的程序[在此](https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py)。

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here

if __name__ == "__main__":
  tf.app.run()

AttributeError: 'module' object has no attribute 'main'

## Intro to Convolutional Neural Networks

Convolutional neural networks(CNNs)是目前处理图像分类任务效果非常好的模型。CNNs应用一系列过滤器应用于图像的原始像素数据来提取和学习高级的特征，模型可以运用提取的高级特征应以图像分类。CNNs包含三个组件：
-  Convolutional layers:应用特定数目的卷积过滤器于图像。对于每个子区域，the layer执行一系列数学操作产生一个值（卷积计算得到一个结果）完成数据特征映射（一个子区域对应一个特征值输出）。卷积层通常将ReLU激活函数应用于输出从而将非线性引入到模型中  。
-  Pooling layers：将卷积层提取的image data进行downsample以减小特征映射的维度以便减少处理时间。常用的池化算法是最大池化，其提取特征映射的子区域，保持子区域的最大值并对其他值。
- Dense(fully connected) layers：全连接层对卷积层提取，pooling layers downsampled的特征执行分类。全连接层中，每个节点都与前一个处理层的节点相连接。
通常，一个CNN模型是由多个执行特征提取的卷积模块组成。每个模块由一个卷积层接着一个pooling layer组成。最后的卷积模块后面跟着一个或多个全连接层以执行分类。CNN模型中最后的全连接层包含模型目标类对应的单节点（模型可能预测的所有可能类），使用softmax激活函数为每个节点（即每个可能的类）生成0-1之间的值（所有节点的softmax值总和为1,即节点的softmax值是每个分类的可能性大小）。我们可以将给定图像的softmax值解释为图像落入每个目标类别的可能性的相对测量值。

## Building the CNN MNIST Classifier

Let's build a model to classify the images in the MNIST dataset using the following CNN architecture:
1. Convolutional Layer #1: Applies 32 5x5 filters (extracting 5x5-pixel subregions), with ReLU activation function
2. Pooling Layer #1: Performs max pooling with a 2x2 filter and stride（步长） of 2 (which specifies that pooled regions do not overlap（重叠）)
3. Convolutional Layer #2: Applies 64 5x5 filters, with ReLU activation function
4. Pooling Layer #2: Again, performs max pooling with a 2x2 filter and stride of 2
5. Dense Layer #1: 1,024 neurons, with dropout regularization rate of 0.4 (probability of 0.4 that any given element will be dropped during training【训练期间任何给定元素将被丢弃的概率为0.4】 )
6. Dense Layer #2 (Logits Layer): 10 neurons, one for each digit target class (0–9).

tf.layers模块包含实现上述三种layer层类型的方法：
- conv2d() :构建二维卷积层，函数参数包含：number of filters,filter kernel size,padding(填充),activation function 
- max_pooling2d(): 构建二维pooling layer使用max-pooling algorithn。函数参数包括：pooling filter size,stride 
- dense():构建全连接层。函数参数包括：number of neurons,activation function

上述的methods以tensor作为输入，并返回一个转换后的tensor作为输出。这使得层与层之间的联系变得更加容易：将前一层的输出tensor作为下一层的输入。

打开cnn_mnist.py并添加如下cnn_model_fn函数，该函数符合TensorFlow's Estimator API的预期接口（更多关于Estimator API的内容在后面创建Estimator中）。cnn_mnist.py将MNIST特征数据，标签和模型模型（TRAIN，EVAL，PREDICT）当作参数；配置CNN并返回预测、代价和训练操作。



In [4]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
   # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

   # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
       inputs=input_layer,
       filters=32,
       kernel_size=[5, 5],
       padding="same",
       activation=tf.nn.relu)
    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)  #梯度优化
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


接下来的内容（每个标题对应其代码块）深入介绍用于创建每个层的tf.layers代码，比如如何计算损失函数，配置训练操作和生成预测结果。如果你已经有了CNN和TensorFlow Estimators的使用经验并且可以直觉地找到上述代码，你可能需要浏览这些部分，或者直接跳到“Training and Evaluating the CNN MNIST Classifier”.

## Input Layer

input layer用于为卷积层和池化层提供二维图像数据。预期数据张量tensor形如[batch_size,image_width,image_height,channels],定义如下：
- `batch_size`:执行梯度下降时批处理的子数据集大小
- `image_width`:示例图片宽度
- `image_height`:示例图片的高度
- `channels`:示例图片的颜色通道数量，对于彩色图像，通道数量是3(红，绿，蓝）；对于单色图像，则只有1个通道(黑色).

这里，MNIST dataset是28*28像素构成的单色图像，因此输入层的输入形状shape应该是[batch_size,28,28,1]

为了将我们的输入特征映射转换成这种shape，我们需要执行下面的reshape 操作：
`input_layer = tf.shape(features["x",[-1,28,28,1])`

In [ ]:
input_layer = tf.shape(features['x'],[-1,28,28,1])

注意，这里我们指定`batch_size`为-1,表示该维度应该根据特征features['x']中输入值的数量动态计算，并保持其他维度的大小不变。这使我们可以将batch_size视为我们可以调整的超参数。例如我们将示例以5批次的形式提供给模型，则特征features['x']包含3920个值[5*28*28=3920](每个图像每个像素都对应一个值），并且input_layer的shape变为[5,28,28,1].类似的，如果以100批次为单位提供示例，则特征feature['x']将包含78400个值,而input_layer的shape变为[100,28,28,1]

## Convolutional Layer #1
在第一个卷积层中，我们对输入层应用32个 5*5 的过滤器filter，使用ReLU激活函数。我们可以勇士conv2d()函数创建卷积层，代码如下：
```
conv1 = tf.layers.conv2d(
    inputs = input_layer,
    filters =32,
    kernel_size=[5,5],
    padding='same',
    activation = tf.nn.relu)
```

参数`inputs`指input tensor,input_tensor的shape必须是[batch_size,image_width,image_height,channels].这里，我们将连接第一个卷积层到`input_layer`,其shape是[batch_size,28,28,1]

参数`filters`指定应用的过滤器的个数，`kernel_size`指定过滤器的维度[`width`,`height`]，`padding`参数指定为两个枚举值中的一个（不区分大小写）：valid(default value) or same.为了指定输出output tensor应该和input tensor具有相同的宽度和高度值，设置参数`padding`=same,这指示TensorFlow将0值添加到input tensor的边缘以保持宽度和高度为28。(without padding,在28*28张量上进行5*5的卷积将产生24*24张量，因为有24*24位置是从28*28的网格中提取5x5得到的）

参数`activation`指定激活函数应用到卷积输出，这里我们指定激活函数为ReLU activation.

conV2d产生的output tensor的shape为[batch_size,28,28,32]:与input tensor具有相同的宽度和高度，但现在有32个通道保存每个过滤器的输出。

## Pooling Layer #1
接下来，我们将刚建立的卷积层连接到第一个pooling layer。我们可以使用`max_pooling2d()` 函数创建池化层执行 max pooling 2x2 filter并设置步长为2:

`pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)`

同样的，`inputs`指定input tensor with a shape of [batch_size,image_width,image_height,channels], 这里，我们的输入张量是`conv1`，`conv1` is the output from the first convolutional layer, which has a shape of [batch_size, 28, 28, 32]

`pool_size`指定最大池过滤器的size[width,height].`strides`指定步长大小。这里我们设置步长为2,这表明过滤器提取的子空间应该在宽度和高度维度上分开2个像素(对于2x2过滤器，这意味着提取区域不重叠)。如果对高度和宽度分别设置不同的步长，则可以改为指定元组或列表（例如，stride = [3，6]）。max_pooling2d（）（pool1）生成的输出张量的形状为[batch_size，14，14，32]：2x2滤镜将宽度和高度分别减少50％。

## Convolutional Layer #2 and Pooling Layer #2
我们可以像之前一样使用conv2d() 和 max_pooling2d()将第二个卷积层和池化层连接到CNN模型。对于卷积层＃2，我们使用ReLU激活配置64个5x5滤波器，并且为了池化层＃2，我们使用与池化层＃1相同的规格（步长为2的2x2最大池化滤波器）
```
conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
```
请注意，卷积层＃2将我们的第一个池层（pool1）的输出张量作为输入，并生成张量conv2作为输出。 conv2的形状为[batch_size，14，14，64]，与pool1的宽度和高度相同（由于padding =“same”），64个通道适用于64个过滤器。

池化层＃2将conv2作为输入，生成pool2作为输出。 pool2的形状为[batch_size，7,7,64]（从conv2减少宽度和高度50％）。

## Dense Layer
接下来，我们将向CNN模型添加dense layer（1024个神经元和ReLU激活函数）对卷积/池化层提取的特征执行分类。在我们连接全连接层之前，我们需要将特征映射（pool2）的shape变为[batch_size,features],因此我们的tensor只有两个维度：

`pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])`

`reshape()`操作中，`-1`表示`batch_size`维度将根据输入数据的数量动态计算。每个示例有(`pool2` width) * 7 (`pool2` height) * 64 (`pool2` channels) features,so we want the features dimension to have a value of 7 * 7 * 64 (3136 in total). The output tensor, pool2_flat, has shape [batch_size, 3136].

现在，我们可以使用 `dense()`来连接全连接层到模型中，代码如下：
`dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)`

The `inputs` argument specifies the input tensor: our flattened feature map, `pool2_flat`. The `units` argument specifies the number of neurons in the dense layer (1,024). The `activation` argument takes the activation function; again, we'll use `tf.nn.relu` to add ReLU activation.

为了改善模型的效果，我们可以在全连接层dense layer中应用dropout regularization，using the `dropout` method in `layers`:
```
dropout = tf.layers.dropout(
    inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
```    
Again, `inputs` specifies the input tensor, which is the output tensor from our dense layer (`dense`).

The `rate` argument specifies the dropout rate; here, we use `0.4`, which means 40% of the elements will be randomly dropped out during training.rate = 0.4意味着40%的节点可能在训练过程中被随机丢弃，弃权。

The `training` argument采用布尔值来确定模型当前是否在训练模型下运行； dropout will only be performed if training is True. Here, we check if the mode passed to our model function cnn\_model\_fn is TRAIN mode.在这里，我们检查传递给模型函数`cnn_model_fn`的`mode`是否是`TRAIN`mode.

Our output tensor dropout has shape [batch_size, 1024].

## Logits Layer
我们的神经网络中最后一层是logits layer,logits layer返回预测的原始值。我们创建一个包含10个神经元（每个神经元依次对应0-9）的全连接层，并使用线性激活（default）：

`logits = tf.layers.dense(inputs=dropout, units=10)`
`logits`is the final output tensor of CNN,has shape[batch_size,10].

## Generate Predicitions
The logits layer of our model returns our predictions as raw values in a [batch_size, 10]-dimensional tensor. Let's convert these raw values into two different formats that our model function can return:
- The predicted class for each example: a digit from 0–9.
- The probabilities for each possible target class for each example: the probability that the example is a 0, is a 1, is a 2, etc.

对一个给定的示例数据，模型的预测结果predict class是具有最高原始值的逻辑张量的对应行中的元素的索引。 我们可以使用`tf.argmax`函数找到这个元素的索引:
`tf.argmax(input=logits, axis=1)`

The `input` argument specifies the tensor from which to extract maximum values—here `logits`. The `axis` argument specifies the axis of the `input` tensor along which to find the greatest value. Here, we want to find the largest value along the dimension with index of 1, which corresponds to our predictions (recall that our logits tensor has shape [batch_size, 10]).

We can derive probabilities from our logits layer by applying softmax activation using `tf.nn.softmax`:

`tf.nn.softmax(logits, name="softmax_tensor")`

注意：我们使用`name`参数来明确命名这个操作`softmax_tensor`，所以我们可以在稍后引用它。 （我们将在“设置日志挂钩”中设置softmax值的日志记录）。

我们用一个字典编译我们的预测，并返回一个`EstimatorSpec`对象：
```
predictions = {
    "classes": tf.argmax(input=logits, axis=1),
    "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
}
if mode == tf.estimator.ModeKeys.PREDICT:
  return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
```

## Calculate Loss
For both training and evaluation, we need to define a `loss function` that measures how closely the model's predictions match the target classes. For multiclass classification problems like MNIST, `cross entropy` is typically used as the loss metric. The following code calculates cross entropy when the model runs in either `TRAIN` or `EVAL` mode,
对于训练和评估，我们需要定义一个损失函数来衡量模型的预测与目标类别的匹配程度。 对于像MNIST这样的多类分类问题，交叉熵通常用作损失度量。 以下代码计算模型在TRAIN或EVAL模式下运行时的交叉熵：:
```
onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
loss = tf.losses.softmax_cross_entropy(
    onehot_labels=onehot_labels, logits=logits)
```    

Let's take a closer look at what's happening above.

Our `labels` tensor contains a list of predictions for our examples, e.g. [1, 9, ...]. In order to calculate cross-entropy, first we need to convert labels to the corresponding one-hot encoding[`label`tensor 是训练数据的标签，因为`logits`的shape为[batch\_size,10],而`labels`shape 为[batch\_size,10]，所以需要对`labels`进行one-hot encoding]:

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 ...]
We use the tf.one\_hot function to perform this conversion. tf.one\_hot() has two required arguments:
- indices. The locations in the one-hot tensor that will have "on values"—i.e., the locations of 1 values in the tensor shown above.表示one-hot tensor 中应该有值的位置。
- depth. The depth of the one-hot tensor—i.e., the number of target classes. Here, the depth is 10.
The following code creates the one-hot tensor for our labels, onehot_labels:
`onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)`

Because `labels` contains a series of values from 0–9, `indices` is just our `labels` tensor, with values cast to integers. The `depth` is 10 because we have 10 possible target classes, one for each digit.

Next, we compute cross-entropy of `onehot\_labels` and the softmax of the predictions from our logits layer. `tf.losses.softmax\_cross\_entropy()` takes `onehot_labels` and `logits` as arguments, performs softmax activation on `logits`, calculates cross-entropy, and returns our loss as a scalar Tensor:

```
loss = tf.losses.softmax_cross_entropy(
    onehot_labels=onehot_labels, logits=logits)
```

## Configure the Training Op
在上一节中，我们定义了CNN模型的损失函数为预测`logits`和labels之间的softmax cross-entropy。让我们在训练过程中减小损失优化模型。我们使用随机梯度下降算法作为优化算法并设置学习率为0.001：
```
if mode == tf.estimator.ModeKeys.TRAIN:
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
  train_op = optimizer.minimize(
      loss=loss,
      global_step=tf.train.get_global_step())
  return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
```

## Add evaluation metrics
To add accuracy metric in our model, we define eval\_metric\_ops dict in EVAL mode as follows:
```
eval_metric_ops = {
    "accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])}
return tf.estimator.EstimatorSpec(
    mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
```

